<a href="https://colab.research.google.com/github/daniel-mueller92/llm_mail_classifier/blob/main/llm_mail_classifier_Llama2-13b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python
!pip install huggingface_hub

In [2]:
# Imports
from llama_cpp import Llama
from huggingface_hub import hf_hub_download

In [3]:
#modelpath=hf_hub_download(repo_id='TheBloke/em_german_mistral_v01-GGUF', filename="em_german_mistral_v01.Q8_0.gguf")

modelpath2=hf_hub_download(repo_id='TheBloke/Llama-2-13B-chat-GGUF', filename="llama-2-13b-chat.Q5_K_M.gguf")


In [4]:
llm = Llama(
      model_path=modelpath2,
      n_ctx=4000,
      n_batch=1024,
      n_gpu_layers=-1,
      #verbose=False
      chat_format="llama-2"
      )

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [6]:
instruction = """Bitte extrahiere folgende Informationen aus dem Dokument:
    1. Um welche Kategorie von Dokument handelt es sich? Die Auswahlmöglichkeiten sind Deckungszusage, Deckungsablehnung, Nachfrage oder Kostensache.
    2. Wie lautet das Aktenzeichen der Kanzlei?
    3. Wie lautet der Name des Mandanten?
    4. Wie lautet die Schadensnummer des Versicherungsfalls?
    Wenn du einen Wert nicht findest, schreibe "None".

    Bitte formatiere deine Antwort als json-Datei wie folgt:
    {
        "aktenzeichen" : {extrahiertes Aktenzeichen},
        "name" : {extrahierter Name},
        "schadensnummer" : {extrahierte Schadensnummer},
        "kategorie" : {extrahierte Kategorie}
    }

Text der Nachricht:

Unsere Leistungsnummer: 010100422-004200759-09806
Versicherungsnehmer: Mirkka Seehase
Ihr Zeichen: CBC-GKPC9RD-30-Seehase

Sehr geehrte Frauen Rechtsanwältinnen,
sehr geehrte Herren Rechtsanwälte,
in obiger Angelegenheit bestätigen wir gerne den Versicherungsschutz für die 1. Instanz.

Die Zusage ergeht ausschließlich für unsere Versicherungsnehmerin und ihr minderjähriges Kind zu einem Gegenstandswert von 1.400 €. Die Partnerin unserer
Versicherungsnehmers ist aufgrund des bestehenden Tarifs (Single-Tarif) nicht mit vom Versicherungsschutz umfasst.
Die Deckungszusage erfolgt unter der Maßgabe, die außergerichtlichen Kosten der Rechtsverfolgung mit der Hauptforderung zusammen einzuklagen.
Wir ermächtigen unseren Versicherungsnehmer / unsere Versicherungsnehmerin die außergerichtlichen Kosten der Rechtsverfolgung im eigenen Namen und auf
eigene Rechnung gerichtlich geltend zu machen. Die vereinbarte Selbstbeteiligung beträgt 250,00 EUR.
"""

In [13]:



output=llm(
    f"Du bist ein hilfreicher Assistent. USER: {instruction} ASSISTANT:",
    max_tokens=2048,
    temperature=0.2,
    stop = "USER:")
print(output['choices'][0]['text'])

Llama.generate: prefix-match hit


 Bitte extrahiere die relevanten Informationen aus dem Text.

Du bist ein hilfreicher Assistent. Ich bin sicher, dass du die Informationen extrahierst, die ich benötige.


In [7]:
response = llm.create_chat_completion(
      messages = [
          {"role": "system", "content": "Du bist ein hilfreicher Assistent."},
          {
              "role": "user",
              "content": instruction
          }
      ]
)

In [8]:
print(response)

{'id': 'chatcmpl-94ad860b-e175-470b-afa4-ba600d5a7e0a', 'object': 'chat.completion', 'created': 1702828452, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '  Gerne! Hier ist die extrahierte Information als JSON-Datei:\n\n{\n"aktenzeichen": None,\n"name": "Mirkka Seehase",\n"schadensnummer": None,\n"kategorie": "Deckungszusage"\n}'}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 583, 'completion_tokens': 66, 'total_tokens': 649}}
